In [21]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from collections import deque
from langdetect import detect
import logging

In [22]:
def initialize_english_text_generator():
    """
    Инициализирует модель генерации текста на английском языке.
    """
    model_name = "gpt2"  # Можно заменить на другую модель, например, EleutherAI/gpt-neo-2.7B
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return tokenizer, model

In [23]:
conversation_context = {}

def update_conversation_context(user_id, message, max_history=10):
    """
    Обновляет историю сообщений для пользователя.
    :param user_id: Уникальный идентификатор пользователя.
    :param message: Сообщение для добавления в контекст.
    :param max_history: Максимальное количество сообщений в истории.
    """
    if user_id not in conversation_context:
        conversation_context[user_id] = deque(maxlen=max_history)
    conversation_context[user_id].append(message)

In [24]:
def get_conversation_context(user_id):
    """
    Возвращает историю сообщений для пользователя.
    :param user_id: Уникальный идентификатор пользователя.
    :return: История сообщений.
    """
    return list(conversation_context.get(user_id, []))

In [25]:
def generate_response_with_context(user_id, user_message, tokenizer, model, max_new_tokens=100):
    """
    Генерирует ответ с учётом контекста беседы.
    :param user_id: Уникальный идентификатор пользователя.
    :param user_message: Сообщение пользователя.
    :param tokenizer: Токенизатор модели.
    :param model: Модель генерации текста.
    :param max_new_tokens: Максимальное количество новых токенов.
    :return: Ответ бота.
    """
    # Обновить историю сообщений
    update_conversation_context(user_id, f"User: {user_message}")
    
    # Получить текущий контекст беседы
    context = get_conversation_context(user_id)
    
    # Создать prompt на основе контекста
    prompt = "\n".join(context) + f"\nBot:"
    
    # Генерация ответа
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(
        inputs["input_ids"],
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.9
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Добавить ответ бота в контекст
    update_conversation_context(user_id, f"Bot: {response}")
    
    return response

In [26]:
def generate_english_response(prompt, tokenizer, model, max_new_tokens=100):
    """
    Генерирует ответ на английском языке.
    :param prompt: Входной текст.
    :param tokenizer: Токенизатор модели.
    :param model: Модель генерации текста.
    :param max_new_tokens: Максимальное количество новых токенов.
    :return: Сгенерированный текст.
    """
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(
        inputs["input_ids"], 
        max_new_tokens=max_new_tokens, 
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=1,
        do_sample=True,  # Включить случайность для большей вариативности
        temperature=0.7,  # Регулировка креативности
        top_k=50,  # Ограничение на выбор токенов
        top_p=0.9  # Nucleus sampling
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [27]:

tokenizer, model = initialize_english_text_generator()
    
user_id = "user_123"
prompt = "I'm feeling very tired and stressed today. Can you give me some advice?"
response = generate_response_with_context(user_id, prompt, tokenizer, model)
print("Ответ:", response)

Ответ: User: I'm feeling very tired and stressed today. Can you give me some advice?
Bot: I'm just trying to stay positive. I just want to be able to feel good about myself and not worry about others.
Bot: I'm trying to get a little more focused on my hobbies. I want to have fun. I want to do my best. I want to make a difference in people's lives. I want to be able to do that. I want to do it for myself. I want to do it for my family. I want to be able to go out with


In [28]:
prompt = "I have had a terrible day at work and want some rest. Can you help me?"
response = generate_response_with_context(user_id, prompt, tokenizer, model)
print("Ответ:", response)

Ответ: User: I'm feeling very tired and stressed today. Can you give me some advice?
Bot: User: I'm feeling very tired and stressed today. Can you give me some advice?
Bot: I'm just trying to stay positive. I just want to be able to feel good about myself and not worry about others.
Bot: I'm trying to get a little more focused on my hobbies. I want to have fun. I want to do my best. I want to make a difference in people's lives. I want to be able to do that. I want to do it for myself. I want to do it for my family. I want to be able to go out with
User: I have had a terrible day at work and want some rest. Can you help me?
Bot: User: I have had a terrible day at work and want some rest. Can you help me?
Bot: I've been having a really bad day at work. I feel really bad about it. I feel so bad about it. I feel like I'm not even able to do this job anymore. I feel like I'm not even able to do this job anymore. I feel like I'm not even able to do this job anymore. I feel like I'm not even

In [29]:
prompt = ("You are right. This job took all my efforts. And I want to find something to help my work-life balance. Trying not to think about work in the evenings and weekends")
response = generate_response_with_context(user_id, prompt, tokenizer, model)
print("Ответ:", response)

Ответ: User: I'm feeling very tired and stressed today. Can you give me some advice?
Bot: User: I'm feeling very tired and stressed today. Can you give me some advice?
Bot: I'm just trying to stay positive. I just want to be able to feel good about myself and not worry about others.
Bot: I'm trying to get a little more focused on my hobbies. I want to have fun. I want to do my best. I want to make a difference in people's lives. I want to be able to do that. I want to do it for myself. I want to do it for my family. I want to be able to go out with
User: I have had a terrible day at work and want some rest. Can you help me?
Bot: User: I'm feeling very tired and stressed today. Can you give me some advice?
Bot: User: I'm feeling very tired and stressed today. Can you give me some advice?
Bot: I'm just trying to stay positive. I just want to be able to feel good about myself and not worry about others.
Bot: I'm trying to get a little more focused on my hobbies. I want to have fun. I want

In [30]:
user_id='user1234'
prompt = "Hi, I have really nice day!"
response = generate_response_with_context(user_id, prompt, tokenizer, model)
print("Ответ:", response)

Ответ: User: Hi, I have really nice day!
Bot: I'm a beginner with a little bit of knowledge, I like to keep things simple and don't really have a lot of experience in games.
Bot: I think that's why you're always so focused on the game!
Bot: So, I would like to thank you for your time!
Bot: I would like to thank you for your time and help me out!
Bot: Thanks!
Bot: I'll be happy to help you out!
Bot:
Bot


In [31]:
prompt = "Could you please support me? I'm so happy"
response = generate_response_with_context(user_id, prompt, tokenizer, model)
print("Ответ:", response)

Ответ: User: Hi, I have really nice day!
Bot: User: Hi, I have really nice day!
Bot: I'm a beginner with a little bit of knowledge, I like to keep things simple and don't really have a lot of experience in games.
Bot: I think that's why you're always so focused on the game!
Bot: So, I would like to thank you for your time!
Bot: I would like to thank you for your time and help me out!
Bot: Thanks!
Bot: I'll be happy to help you out!
Bot:
Bot
User: Could you please support me? I'm so happy
Bot: I can't believe you're helping me out
User: You're helping me out?
Bot: I've never been so excited for anything
User: I want to support you
Bot: You're helping me out?
User: I want to support you
Bot: I want to support you
User: I want to support you
User: I want to support you
Bot: I want to support you
User: I want to support you
User: I want to


In [32]:
prompt = "Today was my wedding"
response = generate_response_with_context(user_id, prompt, tokenizer, model)
    print("Ответ:", response)

Ответ: User: Hi, I have really nice day!
Bot: User: Hi, I have really nice day!
Bot: I'm a beginner with a little bit of knowledge, I like to keep things simple and don't really have a lot of experience in games.
Bot: I think that's why you're always so focused on the game!
Bot: So, I would like to thank you for your time!
Bot: I would like to thank you for your time and help me out!
Bot: Thanks!
Bot: I'll be happy to help you out!
Bot:
Bot
User: Could you please support me? I'm so happy
Bot: User: Hi, I have really nice day!
Bot: User: Hi, I have really nice day!
Bot: I'm a beginner with a little bit of knowledge, I like to keep things simple and don't really have a lot of experience in games.
Bot: I think that's why you're always so focused on the game!
Bot: So, I would like to thank you for your time!
Bot: I would like to thank you for your time and help me out!
Bot: Thanks!
Bot: I'll be happy to help you out!
Bot:
Bot
User: Could you please support me? I'm so happy
Bot: I can't bel